<a href="https://colab.research.google.com/github/VirajMadushan/student-score-prediction/blob/main/CIS6005_Predicting_Student_Test_Scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Student Test Scores
### Kaggle Playground Series – S6E1

**Name:** W.G.Viraj Madushan Jayaweera  
**Student ID:** KD/BSCSD/20/02  
**Module:** CIS6005 – Deep Learning  
**Tool:** Google Colab + Kaggle


## 1. Setup


## 2. Load Data


## 3. Exploratory Data Analysis (EDA)


## 4. Data Preprocessing


## 5. Baseline Model (Machine Learning)


## 6. Deep Learning Model (MLP)


## 7. Model Evaluation and Comparison


## 8. Kaggle Submission


## 9. Save Model for Deployment
